<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/Problema_de_alocacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Instalação do pyomo e do solver para o colab ##
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3

In [3]:
## Importando a biblioteca pyomo ##
import pyomo.environ as pyEnv
import numpy as np

In [5]:
## Modelo matemático ##
## Min x11 + 4x12 + 6x13 + 3x14 + 9x21 + 7x22 + 10x23 + 9x24 + 4x31 + 5x32 + 11x33 + 7x34 +8x41 + 7x42 + 8x43 + 5x44 ##
##                               ##
## S.a x11 + x12 + x13 + x14 = 1 ##
##     x21 + x22 + x23 + x24 = 1 ##
##     x31 + x32 + x33 + x34 = 1 ##
##     x41 + x42 + x43 + x44 = 1 ##
##                               ##
##     x11 + x21 + x31 + x41 = 1 ##
##     x12 + x22 + x32 + x42 = 1 ##
##     x13 + x23 + x33 + x43 = 1 ##
##     x14 + x24 + x34 + x44 = 1 ##

## Dados do problema ##
custos = np.array([[1, 4, 6, 3], [9, 7, 10, 9], [4, 5, 11, 7], [8, 7, 8, 5]])
m = len(custos[0])
n = len(custos[0])

## Declarando o modelo através da biblioteca ##
modelo = pyEnv.ConcreteModel()

## Criando Indices ##
modelo.Indices = list(range(n))

## Criando as variáveis ##
modelo.x = pyEnv.Var(modelo.Indices, modelo.Indices, within = pyEnv.Binary)

## Criando a Função Objetivo ##
def Objetivo(modelo):
  return sum(modelo.x[i,j]*custos[i,j] for i in modelo.Indices for j in modelo.Indices)

modelo.Objetivo = pyEnv.Objective(rule = Objetivo, sense = pyEnv.minimize)

## Criando as restrições ##
def rest1(modelo, i):
  return sum(modelo.x[i,j] for j in modelo.Indices) == 1

def rest2(modelo, j):
  return sum(modelo.x[i,j] for i in modelo.Indices) == 1

modelo.rest1 = pyEnv.Constraint(modelo.Indices, rule = rest1)
modelo.rest2 = pyEnv.Constraint(modelo.Indices, rule = rest2)



In [6]:
## Chamando o Solver ##
solver = pyEnv.SolverFactory('cbc')
result_objetivo = solver.solve(modelo, tee = True)

Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpfakcd715.pyomo.lp -stat=1 -solve -solu /tmp/tmpfakcd715.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 8 (0) rows, 16 (0) columns and 32 (0) elements
Statistics for presolved model
Original problem has 16 integers (16 of which binary)
==== 0 zero objective 10 different
1 variables have objective of 1
1 variables have objective of 3
2 variables have objective of 4
2 variables have objective of 5
1 variables have objective of 6
3 variables have objective of 7
2 variables have objective of 8
2 variables have objective of 9
1 variables have objective of 10
1 variables have objective of 11
==== absolute objective values 10 different
1 variables have objective of 1
1 variables have objective of 3
2 variables have objective of 4
2 variables have objective of 5
1 variables have objective of 6
3 variables have

In [7]:
## Printando o resultado ##
lista = list(modelo.x.keys())
print("Alocação ótima:\n")
for i in lista:
  if modelo.x[i]() != 0:
    print(i, '---', modelo.x[i]())
print()
print('Valor da função objetivo (custo mínimo) =', modelo.Objetivo())

Alocação ótima:

(0, 0) --- 1.0
(1, 2) --- 1.0
(2, 1) --- 1.0
(3, 3) --- 1.0

Valor da função objetivo (custo mínimo) = 21.0
